# Utilities

##### Remove Accents

In [ ]:
def remove_accents(old):
    new = re.sub(r'[àáâãäå]', 'a', old)
    new = re.sub(r'[èéêë]', 'e', new)
    new = re.sub(r'[ìíîï]', 'i', new)
    new = re.sub(r'[òóôõö]', 'o', new)
    new = re.sub(r'[ùúûü]', 'u', new)
    new = re.sub(r'[ñ]', 'n', new)
    return new

##### Pause Code

In [ ]:
def pause_code(start_time='2023-08-09T07:24:30', timezone='EST'):
    est_timezone = pytz.timezone('America/New_York')  # Eastern Standard Time (EST)
    
    # Convert start_time to datetime object in EST timezone
    naive_datetime = datetime.datetime.fromisoformat(start_time)
    est_start_time = est_timezone.localize(naive_datetime)

    # Convert EST time to UTC
    utc_start_time = est_start_time.astimezone(pytz.utc)

    time_difference = utc_start_time - datetime.datetime.now(pytz.utc)
    total_seconds = time_difference.total_seconds()
    
    hours = int(total_seconds // 3600)
    minutes = int((total_seconds % 3600) // 60)
    seconds = int(total_seconds % 60)
    
    est_time_str = est_start_time.strftime("%I:%M%p")
    time_until_str = f"{est_time_str}. {hours} hours, {minutes} minutes, and {seconds} seconds."
    
    print("Time until", time_until_str)

    # Loop with a small sleep interval, checking for interruption
    try:
        while total_seconds > 0:
            time.sleep(1)  # Sleep for 1 second
            total_seconds -= 1
    except KeyboardInterrupt:
        print("Program interrupted by user.")
        return


    ### Set date (may be different in morning)
    # Today's Date
    # YYYY-MM-DD (datetime)
    todaysdate_dt = datetime.date.today()
    
    # YYYY-MM-DD (string)
    todaysdate_dash = str(todaysdate_dt)
    
    # MM/DD/YYYY
    todaysdate_slash = todaysdate_dash.split("-")
    todaysdate_slash = todaysdate_slash[1] + "/" + todaysdate_slash[2] + "/" + todaysdate_slash[0]
    
    # YYYYMMDD
    todaysdate = todaysdate_dash.replace("-", "")
    
    ## MM-DD-YYYY
    todaysdate_dash = todaysdate[:4] + "-" + todaysdate[4:6] + "-" + todaysdate[6:]


    # Get the current date
    current_date = datetime.datetime.now()
    
    # Subtract one day from the current date to get yesterday's date
    yesterday_dt = current_date - datetime.timedelta(days=1)
    
    # Format yesterday's date as "YYYYMMDD"
    yesterdaysdate = yesterday_dt.strftime("%Y%m%d")
    
    # MM/DD/YYYY
    yesterdaysdate_slash = yesterdaysdate[4:6] + "/" + yesterdaysdate[6:8] + "/" + yesterdaysdate[0:4] 
    
    ## MM-DD-YYYY
    yesterdaysdate_dash = yesterdaysdate[:4] + "-" + yesterdaysdate[4:6] + "-" + yesterdaysdate[6:]

##### Identify Pareto-Optimal Observations

In [ ]:
def pareto_optimal(df, objectives, directions):
    data = df[objectives].values
    num_points = data.shape[0]

    # Convert objectives based on direction
    for i, direction in enumerate(directions):
        if direction == "Maximize":
            data[:, i] *= -1

    # Pareto front mask
    pareto_mask = np.ones(num_points, dtype=bool)

    # Check for dominance
    for i in range(num_points):
        for j in range(num_points):
            if i != j:
                # Row j dominates row i if it's better in at least one objective and not worse in others
                if np.all(data[j] <= data[i]) and np.any(data[j] < data[i]):
                    pareto_mask[i] = False
                    break

    # Return the Pareto-optimal rows
    return df[pareto_mask].drop_duplicates()

##### Create Game DataFrame

In [ ]:
def create_games(start_date, end_date, team_dict):
    """
    Fetch game schedules for a given date range.
    
    Parameters:
    - start_date (str): Start date in "YYYYMMDD" format.
    - end_date (str): End date in "YYYYMMDD" format.
    
    Returns:
    - Data Frame: Combined schedule for the specified date range.
    """
    
    # Reformat dates
    start_date = start_date[4:6] + "/" + start_date[6:8] + "/" + start_date[:4] 
    end_date = end_date[4:6] + "/" + end_date[6:8] + "/" + end_date[:4] 

    # Extract year    
    start_year = int(start_date.split("/")[-1])
    end_year = int(end_date.split("/")[-1])
    
    # Initialize an empty list to hold game schedules
    games = []
    
    # Iterate through each year in the range and fetch schedules
    for year in range(start_year, end_year + 1):
        # Determine the bounds for statsapi.schedule
        year_start = start_date if year == start_year else f"01/01/{year}"
        year_end = end_date if year == end_year else f"12/31/{year}"
        
        # Fetch and append the schedules
        games.extend(statsapi.schedule(start_date=year_start, end_date=year_end))
    
    # Create dataframe
    game_df = pd.DataFrame(games)
    # Create date variable
    game_df['date'] = game_df['game_date'].str.replace("-","")
    # Create year variable
    game_df['year'] = game_df['game_date'].str[0:4]
    # Select subsample of games to run (exclude spring training, all-star games, exhibitions, and cancelled games
    game_df = game_df.query('game_type != "S" and game_type != "A" and game_type != "E" and status != "Cancelled" and status != "Postponed"').reset_index(drop=True)

    # Map in team names
    game_df['away_team'] = game_df['away_name'].map(team_dict)
    game_df['home_team'] = game_df['home_name'].map(team_dict)

    # Convert to numeric
    game_df['away_score'] = game_df['away_score'].astype('int')
    game_df['home_score'] = game_df['home_score'].astype('int')
    
    # Drop duplicates
    game_df.drop_duplicates('game_id', inplace=True, keep='last')
    game_df.reset_index(inplace=True, drop=True)
    
    # Drop unnecessary columns
    game_df.drop(columns=['home_pitcher_note', 'away_pitcher_note', 'national_broadcasts', 'series_status', 'summary'], inplace=True)
    
    
    return game_df

##### Create Contest Guide DataFrame

In [ ]:
def create_contests(start_date=None, end_date=None, name=None, entryFee=None, exclusions=['vs', 'Turbo', '@']):
    # Get all file paths
    all_files = glob.glob(os.path.join(baseball_path, "A09. Contest Guides", "*.csv"))

    # Parallel read
    df_list = Parallel(n_jobs=-1)(
        delayed(pd.read_csv)(file, dtype='str') for file in all_files
    )

    # Concatenate
    contest_df = pd.concat(df_list, ignore_index=True)

    # Convert data types
    contest_df['game_id'] = contest_df['game_id'].astype(int)
    contest_df['date'] = pd.to_datetime(contest_df['date'].astype(str), format='%Y%m%d')

    # Apply filters
    if start_date is not None:
        contest_df = contest_df[contest_df['date'] >= pd.to_datetime(start_date, format='%Y%m%d')]
    if end_date is not None:
        contest_df = contest_df[contest_df['date'] <= pd.to_datetime(end_date, format='%Y%m%d')]
    if name is not None:
        contest_df = contest_df[contest_df['name'].str.contains(name)]
    if exclusions != []:
        for exclusion in exclusions:
            contest_df = contest_df[~contest_df['name'].str.contains(exclusion)]

    # Convert date back to string
    contest_df['date'] = contest_df['date'].dt.strftime('%Y%m%d')

    # Calculate slate_size 
    contest_df['slate_size'] = contest_df.groupby('contestKey')['contestKey'].transform('count')

    return contest_df


##### Create Universal Team Map Dictionary

In [ ]:
team_map = pd.read_csv(os.path.join(baseball_path, "Utilities", "Team Map.csv"))

In [ ]:
# Initialize an empty dictionary
team_dict = {}

# Filter columns that end with "TEAM"
team_columns = [col for col in team_map.columns if col.endswith("TEAM") or col.endswith("NAME") or col.endswith("Id")]

# Iterate over each row in the dataframe
for _, row in team_map.iterrows():
    bbref_team = row['BBREFTEAM']  # Get the BBREFTEAM value
    # Iterate over filtered columns in the row
    for column in team_columns:
        value = row[column]
        if pd.notna(value):  # Skip NaN values
            team_dict[value] = bbref_team

##### Create Venue Map DataFrame

In [ ]:
def create_venue_map(write=False):
    # Fetch JSON data from the URL
    response = requests.get(url)
    data = response.json()
    
    # Extract venue details 
    venues = data.get("venues", data)  
    
    # Normalize the JSON into a DataFrame
    df = pd.json_normalize(venues)
    
    # Save to CSV
    if write == True:
        df.sort_values('id').to_csv(os.path.join(baseball_path, "Utilities", "Venue Map.csv"), index=False)


    return df

##### Add Missing Values

In [ ]:
# Read in Venue Map
venue_map_df = pd.read_csv(os.path.join(baseball_path, "Utilities", "Venue Map.csv"))

# George M. Steinbrenner
venue_map_df.loc[venue_map_df['id'] == 2523, ['fieldInfo.leftCenter', 'fieldInfo.rightCenter']] = [399.0, 385.0] # Yankee Stadium dimensions
# Sutter Health Park
venue_map_df.loc[venue_map_df['id'] == 2529, ['fieldInfo.leftCenter', 'fieldInfo.rightCenter']] = [375.0, 368.0] # https://x.com/JonPgh/status/1875224135573594599